# Packages

The cell below contains the packages that are used in the code.

In [1]:
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer
from IconicityModel import *

# workaround for the tornado 5.0 bug
import nest_asyncio
nest_asyncio.apply()

# Agent representation

This section contains code about the agent representation. Agents will be represented via circles. The colour and whether it is a filled circle or not depends on the properties of the agent.

The radius of the circles indicate the age of the agent. A smaller circle represents an agent of age 0 and a bigger circle represents an agent of age 1. The colour indicates the age of acquisition of the agent; green stands for L1 signers and red for L2 signers.

In [2]:
def agent_representation(agent):
    iconicity_level = round(agent.iconicity_ratio() * 100)
    representation = {
                      # shape and colour of the agent
                      "Shape": "circle",
                      "Filled": "true",
                      "r": 0.5,
                      
                      # visible properties in visualisation
                      "Age of acquisition": agent.aoa,
                      "Age": agent.age,
                      "Avg. iconicity level": str(iconicity_level) + "%",
        
                      # drawing layer
                      "Layer": 0,
                      }
    
    if agent.age == 0:
        representation["r"] = 0.3
    elif agent.age == 1:
        representation["r"] = 0.5
        
    if agent.aoa == "L1":
        representation["Color"] = "green"
    elif agent.aoa == "L2":
        representation["Color"] = "red"

    return representation

# Grid representation

This section contains code about the grid representation.

In [3]:
grid = CanvasGrid(agent_representation, 10, 10, 500, 500)

# Server representation

This section contains code about the server representation.

In [ ]:
server = ModularServer(IconicityModel,
                       [grid],
                       "Iconicity Model",
                       {"width":10, "height":10, "vocab_size":10, "word_length":5, "initial_degree_of_iconicity":0.8, "learning_error":1})
server.port = 8521
server.launch()

Interface starting at http://127.0.0.1:8521
Socket opened!
{"type":"reset"}
{"type":"get_step","step":1}
{"type":"get_step","step":2}
